In [8]:
import os
import uuid
from typing import List

from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import openai
from agno.agent import Agent
from agno.vectordb.qdrant import Qdrant as AgnoQdrant
from agno.knowledge import AgentKnowledge
from agno.embedder.openai import OpenAIEmbedder
from langchain_core.messages import HumanMessage
from agno.document.chunking.document import DocumentChunking

In [ ]:
import asyncio
from agno.agent import Agent
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase
from agno.vectordb.qdrant import Qdrant

COLLECTION_NAME = "Akreage"

# Initialize Qdrant with local instance
vector_db = Qdrant(
    collection=COLLECTION_NAME, 
    url="http://localhost:6333"
)

# Create knowledge base
knowledge_base = PDFUrlKnowledgeBase(
    urls=["https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf"],
    vector_db=vector_db,
)

agent = Agent(knowledge=knowledge_base, show_tool_calls=True)

if __name__ == "__main__":
    # Load knowledge base asynchronously
    asyncio.run(knowledge_base.aload(recreate=False))  # Comment out after first run

    # Create and use the agent asynchronously
    asyncio.run(agent.aprint_response("How to make Tom Kha Gai", markdown=True))

In [15]:
import os
from typing import List, Union
from pathlib import Path

from agno.agent import Agent
from agno.knowledge.pdf import PDFKnowledgeBase
from agno.knowledge.combined import CombinedKnowledgeBase
from agno.vectordb.qdrant import Qdrant
from agno.models.openai import OpenAIChat
from agno.embedder.openai import OpenAIEmbedder
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
class MultiPDFQdrantAgent:
    """
    A class to handle multiple PDF files, embed them in Qdrant vector database,
    and provide an interactive agent interface.
    """
    
    def __init__(
        self,
        pdf_paths: List[Union[str, Path]],
        collection_name: str = "multi_pdf_collection",
        qdrant_url: str = "http://localhost:6333",
        model_name: str = "gpt-4o-mini",
        embedding_model: str = "text-embedding-3-small"
    ):
        """
        Initialize the Multi-PDF Qdrant Agent.
        
        Args:
            pdf_paths: List of paths to PDF files
            collection_name: Name of the Qdrant collection
            qdrant_url: URL of the Qdrant instance
            model_name: OpenAI model name for the agent
            embedding_model: OpenAI embedding model name
        """
        self.pdf_paths = [Path(path) for path in pdf_paths]
        self.collection_name = collection_name
        self.qdrant_url = qdrant_url
        self.model_name = model_name
        self.embedding_model = embedding_model
        
        # Validate PDF paths
        self._validate_pdf_paths()
        
        # Initialize components
        self.vector_db = None
        self.knowledge_base = None
        self.agent = None
        
    def _validate_pdf_paths(self):
        """Validate that all PDF paths exist and are PDF files."""
        for pdf_path in self.pdf_paths:
            if not pdf_path.exists():
                raise FileNotFoundError(f"PDF file not found: {pdf_path}")
            if pdf_path.suffix.lower() != '.pdf':
                raise ValueError(f"File is not a PDF: {pdf_path}")
    
    def _setup_vector_db(self):
        """Initialize the Qdrant vector database."""
        print(f"Setting up Qdrant vector database...")
        self.vector_db = Qdrant(
            collection=self.collection_name,
            url=self.qdrant_url,
            embedder=OpenAIEmbedder()
        )
        print(f"✓ Qdrant vector database initialized with collection: {self.collection_name}")
    
    def _setup_knowledge_base(self):
        """Create knowledge base from multiple PDF files."""
        print(f"Setting up knowledge base from {len(self.pdf_paths)} PDF files...")
        
        knowledge_bases = []
        
        for i, pdf_path in enumerate(self.pdf_paths):
            print(f"Processing PDF {i+1}/{len(self.pdf_paths)}: {pdf_path.name}")
            
            # Create individual knowledge base for each PDF
            pdf_kb = PDFKnowledgeBase(
                path=str(pdf_path),
                vector_db=self.vector_db,
                # Optional: Add custom reader settings
                # reader=PDFReader(chunk_size=3000, chunk_overlap=200)
            )
            knowledge_bases.append(pdf_kb)
        
        # Combine all knowledge bases if multiple PDFs
        if len(knowledge_bases) == 1:
            self.knowledge_base = knowledge_bases[0]
        else:
            self.knowledge_base = CombinedKnowledgeBase(
                sources=knowledge_bases,
                vector_db=self.vector_db
            )
        
        print("✓ Knowledge base created successfully")
    
    def _setup_agent(self):
        """Initialize the agent with the knowledge base."""
        print("Setting up agent...")
        self.agent = Agent(
            model=OpenAIChat(id=self.model_name),
            knowledge=self.knowledge_base,
            show_tool_calls=True,
            instructions=[
                "You are a helpful assistant that can answer questions based on the provided PDF documents.",
                "Always cite the source when providing information from the documents.",
                "If you cannot find the answer in the documents, clearly state that.",
                "Be concise but thorough in your responses."
            ]
        )
        print("✓ Agent initialized successfully")
    
    def setup(self, recreate_embeddings: bool = False):
        """
        Setup the complete system: vector DB, knowledge base, and agent.
        
        Args:
            recreate_embeddings: If True, recreate embeddings even if they exist
        """
        print("Starting Multi-PDF Qdrant Agent setup...")
        
        # Setup vector database
        self._setup_vector_db()
        
        # Setup knowledge base
        self._setup_knowledge_base()
        
        # Load knowledge base (create embeddings)
        print("Loading knowledge base and creating embeddings...")
        self.knowledge_base.load(recreate=recreate_embeddings)
        print("✓ Knowledge base loaded and embeddings created")
        
        # Setup agent
        self._setup_agent()
        
        print("🎉 Multi-PDF Qdrant Agent setup complete!")
    
    def query(self, question: str, markdown: bool = True) -> str:
        """
        Query the agent with a question.
        
        Args:
            question: The question to ask
            markdown: Whether to format response as markdown
            
        Returns:
            The agent's response
        """
        if self.agent is None:
            raise RuntimeError("Agent not initialized. Call setup() first.")
        
        print(f"\n🤔 Question: {question}")
        print("-" * 50)
        
        response = self.agent.print_response(question, markdown=markdown)
        return response
    
    def interactive_chat(self):
        """Start an interactive chat session."""
        if self.agent is None:
            raise RuntimeError("Agent not initialized. Call setup() first.")
        
        print("\n🤖 Interactive Chat Started!")
        print("Type 'quit', 'exit', or 'bye' to end the session.")
        print("=" * 50)
        
        while True:
            try:
                question = input("\n💬 You: ").strip()
                
                if question.lower() in ['quit', 'exit', 'bye']:
                    print("👋 Goodbye!")
                    break
                
                if not question:
                    continue
                
                self.query(question)
                
            except KeyboardInterrupt:
                print("\n👋 Goodbye!")
                break
            except Exception as e:
                print(f"❌ Error: {e}")
    
    def get_collection_info(self):
        """Get information about the Qdrant collection."""
        if self.vector_db is None:
            return "Vector database not initialized"
        
        return {
            "collection_name": self.collection_name,
            "qdrant_url": self.qdrant_url,
            "pdf_count": len(self.pdf_paths),
            "pdf_files": [p.name for p in self.pdf_paths]
        }


# Example usage and main function
def main():
    """Example usage of the Multi-PDF Qdrant Agent."""
    
    # List of PDF paths to process
    pdf_paths = [
        "C:\documents\GEN AI\Agno\Agno-Framework\XO-testing\Akreage Launch Pad Whitepaper.pdf",
        "C:\documents\GEN AI\Agno\Agno-Framework\XO-testing\Akreage_Launchpad_Scope_of_Work (1).pdf",
        "C:\documents\GEN AI\Agno\Agno-Framework\XO-testing\Plan of Action .pdf",
        # Add more PDF paths as needed
    ]
    
    # Create the agent
    agent = MultiPDFQdrantAgent(
        pdf_paths=pdf_paths,
        collection_name="my_documents",
        qdrant_url="http://localhost:6333",  # Make sure Qdrant is running
        model_name="gpt-4o-mini",
        embedding_model="text-embedding-3-small"
    )
    
    try:
        # Setup the agent (this will create embeddings)
        agent.setup(recreate_embeddings=False)  # Set to True to recreate embeddings
        
        # Print collection info
        print("\n📊 Collection Info:")
        info = agent.get_collection_info()
        for key, value in info.items():
            print(f"  {key}: {value}")
        
        # Example queries
        example_queries = [
            "What are the main topics covered in these documents?",
            "Can you summarize the key points from all the documents?",
            "What specific information is available about [your topic]?"
        ]
        
        print("\n🔍 Example Queries:")
        for query in example_queries:
            print(f"  - {query}")
        
        # Start interactive chat
        agent.interactive_chat()
        
    except Exception as e:
        print(f"❌ Error: {e}")
        print("Make sure:")
        print("1. Qdrant is running on http://localhost:6333")
        print("2. OpenAI API key is set in environment variables")
        print("3. All PDF paths are valid")


if __name__ == "__main__":
    # Set your OpenAI API key
    # os.environ["OPENAI_API_KEY"] = "your-api-key-here"
    
    # Run the main function
    main()

Starting Multi-PDF Qdrant Agent setup...
Setting up Qdrant vector database...
✓ Qdrant vector database initialized with collection: my_documents
Setting up knowledge base from 3 PDF files...
Processing PDF 1/3: Akreage Launch Pad Whitepaper.pdf
Processing PDF 2/3: Akreage_Launchpad_Scope_of_Work (1).pdf
Processing PDF 3/3: Plan of Action .pdf
✓ Knowledge base created successfully
Loading knowledge base and creating embeddings...


INFO Loading knowledge base

INFO Reading: Akreage Launch Pad Whitepaper

INFO Added 0 documents to knowledge base

INFO Reading: Akreage_Launchpad_Scope_of_Work (1)

...

✓ Knowledge base loaded and embeddings created
Setting up agent...
✓ Agent initialized successfully
🎉 Multi-PDF Qdrant Agent setup complete!

📊 Collection Info:
  collection_name: my_documents
  qdrant_url: http://localhost:6333
  pdf_count: 3
  pdf_files: ['Akreage Launch Pad Whitepaper.pdf', 'Akreage_Launchpad_Scope_of_Work (1).pdf', 'Plan of Action .pdf']

🔍 Example Queries:
  - What are the main topics covered in these documents?
  - Can you summarize the key points from all the documents?
  - What specific information is available about [your topic]?

🤖 Interactive Chat Started!
Type 'quit', 'exit', or 'bye' to end the session.

🤔 Question: what is Akreage
--------------------------------------------------


Output()


🤔 Question: what is the plan of action
--------------------------------------------------


Output()

👋 Goodbye!


In [1]:
from agno.agent import Agent
from agno.knowledge.pdf import PDFKnowledgeBase, PDFReader
from agno.vectordb.qdrant import Qdrant
from agno.document.chunking.agentic import AgenticChunking
# from agno.embedder.sentence_transformer import SentenceTransformerEmbedder
from agno.models.openai import OpenAIChat
from agno.document.chunking.recursive import RecursiveChunking

COLLECTION_NAME = "multipleDocuments"

vector_db = Qdrant(collection=COLLECTION_NAME, url="http://localhost:6333",

)

# Create a knowledge base with the PDFs from the data/pdfs directory
knowledge_base = PDFKnowledgeBase(
    path="data/",
    vector_db=vector_db,
    reader=PDFReader(chunk=True),
    chunking_strategy=RecursiveChunking(chunk_size=5000,overlap=500),
)

# Create an agent with the knowledge base
agent = Agent(
    model=OpenAIChat(id="gpt-4.1-nano"),
    knowledge=knowledge_base,
    search_knowledge=True,
    instructions=[
                "You are a helpful assistant that can answer questions based on the provided PDF documents.",
                "Always cite the source when providing information from the documents.",
                "If you cannot find the answer in the documents, clearly state that.",
                "Be concise but thorough in your responses."
            ]
)

if __name__ == "__main__":
    # Comment out after first run
    
    knowledge_base.load(recreate=False)

INFO Embedder not provided, using OpenAIEmbedder as default.

INFO Creating collection

INFO Loading knowledge base

INFO Reading: Akreage Launch Pad Whitepaper

INFO Added 6 documents to knowledge base

...

INFO Added 2 documents to knowledge base

INFO Reading: Introduction

INFO Added 19 documents to knowledge base

INFO Reading: Plan of Action

INFO Added 5 documents to knowledge base

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')


In [6]:

agent.run("whats LATs tree search").content

'LATs tree search is a research framework that combines classical AI search methods, specifically Monte Carlo Tree Search (MCTS), with Large Language Model (LLM)-based reasoning. The core idea is for the agent to explore multiple potential action sequences in a tree structure rather than following a single linear thought process. At each decision point, LATs simulates various "what if" scenarios using the LLM to predict outcomes, evaluates these branches, and then selects the most promising one to execute. It also incorporates reflection at each node to identify and correct errors or improve reasoning, making it a powerful approach for complex reasoning problems. This method was reported to outperform simpler agents in challenging tasks such as math or coding problems, but it is resource-intensive due to the multiple LLM calls involved (source: knowledge base, pages 9 and 19).'

In [2]:
from agno.agent import Agent
from agno.knowledge.pdf import PDFKnowledgeBase, PDFReader
from agno.vectordb.qdrant import Qdrant
from agno.document.chunking.recursive import RecursiveChunking
from agno.models.openai import OpenAIChat
from pathlib import Path
from agno.knowledge import AgentKnowledge
from agno.embedder.openai import OpenAIEmbedder

def create_user_agent(user_id: str, user_pdf_dir: str):
    vector_db = Qdrant(
        collection="shared_pdf_index",  # Shared collection
        url="http://localhost:6333"
    )

    knowledge_base = PDFKnowledgeBase(
        path=user_pdf_dir,
        vector_db=vector_db,
        reader=PDFReader(chunk=True),
        # chunking_strategy=RecursiveChunking(chunk_size=800, overlap=100),
        metadata={"user_id": user_id}  # chunks tagged during ingestion
    )

    agent = Agent(
        model=OpenAIChat(id="gpt-4.1-nano"),  # fast model
        knowledge=knowledge_base,
        search_knowledge=True,
        user_id=user_id,
        instructions=[
            "You are a helpful assistant that can answer questions based on the provided PDF documents.",
            "Always cite the source when providing information from the documents.",
            "If you cannot find the answer in the documents, clearly state that.",
            "Be concise but thorough in your responses."
        ]
    )

    return agent, knowledge_base


In [3]:
if __name__ == "__main__":
    user_id = "user_abc"
    user_folder = f"data/{user_id}"  # e.g., user uploads go here

    agent, kb = create_user_agent(user_id, user_folder)

    # Load docs once (or only when new ones are added)
    kb.load(recreate=False)

    while True:
        query = input("Ask your assistant: ")
        if query.lower() in {"exit", "quit"}:
            break
        response = agent.run(query)
        print("🤖", response.content)


INFO Embedder not provided, using OpenAIEmbedder as default.

INFO Loading knowledge base

INFO Reading: Akreage Launch Pad Whitepaper

INFO Added 6 documents to knowledge base

INFO Reading: Akreage_Launchpad_Scope_of_Work (1)

INFO Added 2 documents to knowledge base

INFO Reading: Plan of Action

INFO Added 5 documents to knowledge base

🤖 The plan for Week 1 involves finalizing the requirements and preparing the technical groundwork. Specifically, it includes:

- Finalizing the MVP scope and user flows, such as user interactions, milestone release flow, roles, and NFTs needed for governance.
- Deciding on the blockchain network (e.g., Polygon, Ethereum, etc.)
- Drafting the Supabase schema for project metadata, users, milestones, and investments.
- Outlining how prompts in XO will be used to generate front-end components.

This initial week is focused on setting a solid foundation for the development phases to follow.
🤖 Week 8 is designated as a buffer and final development tasks period. The key activities include:

- Addressing any unforeseen complexities from previous weeks, such as UX refinements and minor smart contract adjustments.
- Performing additional internal testing to verify all end-to-end user flows.
- Optimizing the front-end code and finalizing gas optimizations for smart contracts.
- Updating documenta